##### All-Star/NBA Predictions

MLP implementation with Tensorflow and Keras

In [181]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [4]:
fileName = 'C:/Users/Roger/Documents/GitHub/All-Star-Predictions/baseData/ML/all_stats_20211201.csv'
df = pd.read_csv(fileName)
print('All players data loaded.')

X = df[['RPG','APG','SBPG','PPG','TS','WS48','Perc']]
y = df['allLeague']

All players data loaded.


In [313]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)



In [326]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


#X_train = tf.keras.utils.normalize(X_train, axis=1)
#X_test = tf.keras.utils.normalize(X_test, axis=1)

In [344]:
model = tf.keras.models.Sequential()

model.add(Dense(4, activation='relu', name = "hidden1"))

#model.add(Dense(2, activation='relu', name = "hidden1"))
#model.add(Dense(2, activation='relu', name = "hidden2"))
model.add(Dense(1, activation='sigmoid', name = "sigmoid"))

In [345]:
model.compile(optimizer='adam',
             loss="binary_crossentropy",
             metrics='accuracy')

In [346]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
186/186 [==============================] - 0s 460us/step - loss: 0.6551 - accuracy: 0.6552
Epoch 2/100
186/186 [==============================] - 0s 444us/step - loss: 0.4183 - accuracy: 0.8432
Epoch 3/100
186/186 [==============================] - 0s 444us/step - loss: 0.3182 - accuracy: 0.8956
Epoch 4/100
186/186 [==============================] - 0s 482us/step - loss: 0.2623 - accuracy: 0.9184
Epoch 5/100
186/186 [==============================] - 0s 482us/step - loss: 0.2214 - accuracy: 0.9336
Epoch 6/100
186/186 [==============================] - 0s 471us/step - loss: 0.2015 - accuracy: 0.9262
Epoch 7/100
186/186 [==============================] - 0s 460us/step - loss: 0.1840 - accuracy: 0.9270
Epoch 8/100
186/186 [==============================] - 0s 482us/step - loss: 0.1707 - accuracy: 0.9337
Epoch 9/100
186/186 [==============================] - 0s 449us/step - loss: 0.1575 - accuracy: 0.9387
Epoch 10/100
186/186 [==============================] - 0s 460us/step - l

186/186 [==============================] - 0s 512us/step - loss: 0.1334 - accuracy: 0.9443
Epoch 80/100
186/186 [==============================] - 0s 465us/step - loss: 0.1483 - accuracy: 0.9378
Epoch 81/100
186/186 [==============================] - 0s 465us/step - loss: 0.1509 - accuracy: 0.9350
Epoch 82/100
186/186 [==============================] - 0s 703us/step - loss: 0.1412 - accuracy: 0.9424
Epoch 83/100
186/186 [==============================] - 0s 633us/step - loss: 0.1412 - accuracy: 0.9422
Epoch 84/100
186/186 [==============================] - 0s 617us/step - loss: 0.1468 - accuracy: 0.9352
Epoch 85/100
186/186 [==============================] - 0s 476us/step - loss: 0.1365 - accuracy: 0.9431
Epoch 86/100
186/186 [==============================] - 0s 460us/step - loss: 0.1432 - accuracy: 0.9421
Epoch 87/100
186/186 [==============================] - 0s 465us/step - loss: 0.1521 - accuracy: 0.9352
Epoch 88/100
186/186 [==============================] - 0s 471us/step - loss:

In [347]:
val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss, val_acc)

47/47 [==============================] - 0s 457us/step - loss: 0.1431 - accuracy: 0.9434
0.14305517077445984 0.9434343576431274


In [351]:
y_true, y_pred = y_test, model.predict(X_test)

y_pred_bin = np.zeros([len(y_pred),1], dtype=int)

In [349]:
for index, item in enumerate(y_pred):
    #print(item)
    if item > 0.5:
        y_pred_bin[index] = [1]
    else:
        y_pred_bin[index] = [0]
        
print(confusion_matrix(y_true, y_pred_bin))

[[1252   25]
 [  59  149]]


In [350]:
print(classification_report(y_true, y_pred_bin))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      1277
           1       0.86      0.72      0.78       208

    accuracy                           0.94      1485
   macro avg       0.91      0.85      0.87      1485
weighted avg       0.94      0.94      0.94      1485



In [324]:
df_2022 = pd.read_csv("C:/Users/Roger/Documents/GitHub/All-Star-Predictions/baseData/dailystats/2022-01-06/stats_20220106.csv")
X_2022 = df_2022[['RPG','APG','SBPG','PPG','TS','WS48','Perc']]
#X_2022 = tf.keras.utils.normalize(X_2022, axis=1)

In [325]:
df_2022pred = df_2022.copy()
X_predicted = model.predict(X_2022)
df_2022pred['Prob'] = [i[0] for i in X_predicted]
df_2022pred

,Player,Year,G,MP,MPG,RPG,APG,SBPG,PPG,TS,rTS,WS48,Perc,Prob
0,Precious Achiuwa,2022,26,641,25.1,7.8,1.6,1.1,8.1,0.439,-0.118,0.039,0.514,0.003386
1,Steven Adams,2022,39,1004,26.2,9.4,3.0,1.4,7.2,0.568,0.011,0.148,0.641,0.006288
2,Nickeil Alexander-Walker,2022,35,968,28.2,3.8,2.6,1.3,13.8,0.464,-0.093,-0.013,0.342,0.005368
3,Grayson Allen,2022,36,1023,28.9,3.4,1.4,1.2,12.1,0.572,0.015,0.107,0.625,0.002272
4,Jarrett Allen,2022,31,1016,33.4,10.9,1.9,2.4,17.4,0.720,0.163,0.258,0.553,0.163855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,Kenrich Williams,2022,34,708,21.2,4.0,1.9,1.1,7.2,0.534,-0.023,0.096,0.351,0.000594
191,Robert Williams,2022,29,829,29.1,9.1,1.8,2.7,10.0,0.737,0.180,0.224,0.474,0.007248
192,Christian Wood,2022,36,1112,31.5,10.6,2.2,1.6,17.3,0.556,-0.001,0.091,0.282,0.102544
193,Trae Young,2022,33,1143,35.3,4.0,9.7,1.0,28.9,0.588,0.031,0.179,0.459,0.621039
